题目相关参数

- 一次绕组星形联结
- 二次绕组三角形联结

In [1]:
import numpy as np 

S_N = np.float(8000) # kVA 额定功率 8MVA
P_L = np.float(4000) # kVA 负载有功额定 4MVA
Q_L = np.float(1000) # kVA 负载无功额定 1MVA

U_1N = np.float(110) # kV 一次侧额定电压 110kV
U_2N = np.float(6.3) # kV 二次侧额定电压 6.3kV
U_1N_phi = U_1N / np.sqrt(3) # 一次侧相电压 星形联结
U_2N_phi = U_2N / np.sqrt(3) # 二次侧相电压 星形联结
k = U_1N_phi / U_2N_phi  # 变比

I_1N_phi = S_N / (3 * U_1N_phi) # 一次侧额定相流
I_2N_phi = S_N / (3 * U_2N_phi)

R_line = 1 # ohm 线路电阻
L_line = 0.001 # H 线路电感

p_0 = np.float(12) # kW
f = np.float(50) # 频率
pi = np.pi
p_kN = np.float(50) # kW

Z_k = [9.453,158.5] # 短路阻抗 第一个元素为电阻 第二个元素为电抗 折算到一次侧
Z_m = [np.float(31400),np.float(175148)] # 励磁阻抗 电阻、电抗

将一二次绕组的电抗换算为电感

In [2]:
L_m = Z_m[1] / (2*pi*f) # 励磁电感
# 根据 R_1 + k^2 R_2 = R_k X_1 + k^2 X_2 = X_k
Z_k_1 = [Z_k[0]/(1+np.square(k)),Z_k[1]/(1+np.square(k))] # 一次绕组的电阻和电抗 
Z_k_2 = [Z_k[0]/(1+np.square(k)),Z_k[1]/(1+np.square(k))] # 二次绕组的电阻和电抗
L_1_m = Z_k_1[1] / (2*pi*f) # 一次绕组的电感
L_2_m = Z_k_2[1] / (2*pi*f) #  二次绕组的电感
print('一次绕组的电阻为{}ohm 电感为{}H'.format(Z_k_1[0],L_1_m))
print('二次绕组的电阻为{}ohm 电感为{}H'.format(Z_k_2[0],L_2_m))
print('励磁绕组的电阻为{}ohm 电感为{}H'.format(Z_m[0],L_m))

一次绕组的电阻为0.030906025606914186ohm 电感为0.0016495021883982152H
二次绕组的电阻为0.030906025606914186ohm 电感为0.0016495021883982152H
励磁绕组的电阻为31400.0ohm 电感为557.5133994531857H


实验数据

In [3]:
U_2 = 3230 * np.sqrt(3) # 二次侧电压
voltage_down = (U_2N * 1000 - U_2) / (U_2N * 1000) # 压降
P_2 = np.float(3155) # 负载有功 测量值 kVA
Q_2 = np.float(788.8) # 负载无功 测量值 kVA
P_1 = np.float(3889) # 输入有功 测量值 kVA
Q_1 = np.float(1134) # 输入无功 测量值 kVA
eta = P_2 / P_1 
print('二次侧负载的相电压为{}kV'.format(U_2/np.sqrt(3)))
print('压降为{}%'.format(voltage_down*100))
print('效率为{}%'.format(eta*100))

二次侧负载的相电压为3230.0000000000005kV
压降为11.198030024643273%
效率为81.12625353561327%


计算负载的阻抗 并折算到一次侧 注意 负载的RL是并联

In [4]:
R_L = np.square(U_2N_phi*1000)/((P_L/3)*1000) # 负载电阻
X_L = np.square(U_2N_phi*1000)/((Q_L/3)*1000) # 负载电抗
revise_coff = (R_L*X_L)/(np.sum(np.square([R_L,X_L])))  # 负载阻抗修正系数 是并联
# 负载阻抗 折算到一次侧
Z_L = [revise_coff * X_L * np.square(k), revise_coff * R_L * np.square(k)]
# 加上线路阻抗后折算到一次侧
Z_L_plus_line = [Z_L[0]+R_line*np.square(k),Z_L[1]+2*pi*f*L_line*np.square(k)]
# 加上一二次绕组的阻抗
Z_total = list(np.array(Z_L_plus_line) + np.array(Z_k))

计算一次相电流 注意 是在一相进行计算 忽略励磁

之后再计算负载的相电压

In [5]:
I_1_phi = U_1N_phi * 1000 / np.sqrt(np.sum(np.square(Z_total)))
# 折算回二次侧
U_2_phi_cal = I_1_phi * np.sqrt(np.sum(np.square(Z_L))) / k
print('手算得二次侧负载相电压{}V'.format(U_2_phi_cal))
print('仿真得二次侧负载相电压{}V'.format(U_2/np.sqrt(3)))

手算得二次侧负载相电压3229.088738927877V
仿真得二次侧负载相电压3230.0000000000005V


计算输出容量

In [6]:
S_L_cal = 3 * I_1_phi * k * U_2_phi_cal
cos_phi = Z_L[0] / np.sqrt(np.sum(np.square(Z_L)))
print('计算得输出容量为{}kVA'.format(S_L_cal/1000))
print('计算得输出有功为{}kVA'.format(S_L_cal*cos_phi))
print('仿真得输出容量为{}kVA'.format(np.sqrt(np.sum(np.square([P_2,Q_2])))))
print('仿真得输出有功为{}kVA'.format(P_2))

计算得输出容量为3249.5601230236116kVA
计算得输出有功为3152536.3821226987kVA
仿真得输出容量为3252.1116893489375kVA
仿真得输出有功为3155.0kVA
